# B-Trees & Advanced Table Indexing

We'll be exploring advanced table indexing through the B-tree index. 

The B-tree index is a commonly used data structure in many applications. It is a balanced tree with a root node and multiple nodes below it. The root node splits the range of values in the index column, and each side of the tree has a subtree. 

Values less than the node value are stored on the left branch, and values greater than the node value are stored on the right branch. This pattern continues until we reach the bottom of the tree. 

- B-Trees are the most common type of index
- They're used when here's a large number of possible values in a column (also called high cardinality)
- The B-tree rebalances as needed to keep the depth of the tree about the same for all paths (preventing a lopsided tree, which would make it slower on one side)
- When looking up a value, the time will be proportional to the log of the number of nodes in the tree (depth)

In [5]:
#Let's visualize a b-tree with code
from graphviz import Digraph

class BTreeNode:
    def __init__(self, t, leaf=True):
        self.keys = []
        self.children = []
        self.leaf = leaf
        self.t = t

class BTree:
    def __init__(self, t):
        self.root = None
        self.t = t
        self.degree = 2 * t

    def insert(self, key):
        if self.root is None:
            self.root = BTreeNode(self.t)
            self.root.keys.append(key)
        else:
            if len(self.root.keys) == (2 * self.t) - 1:
                new_root = BTreeNode(self.t, leaf=False)
                new_root.children.append(self.root)
                self.split_child(new_root, 0)
                self.root = new_root
            self._insert_non_full(self.root, key)

    def _insert_non_full(self, node, key):
        i = len(node.keys) - 1
        while i >= 0 and key < node.keys[i]:
            i -= 1
        if node.leaf:
            node.keys.insert(i + 1, key)
        else:
            if len(node.children[i + 1].keys) == (2 * self.t) - 1:
                self.split_child(node, i + 1)
                if key > node.keys[i + 1]:
                    i += 1
            self._insert_non_full(node.children[i + 1], key)

    def split_child(self, node, index):
        y = node.children[index]
        z = BTreeNode(self.t, leaf=y.leaf)
        z.keys = y.keys[self.t:]
        y.keys = y.keys[:self.t]
        if not y.leaf:
            z.children = y.children[self.t:]
            y.children = y.children[:self.t]
        node.keys.insert(index, y.keys.pop())
        node.children.insert(index + 1, z)

    def visualize(self):
        dot = Digraph()
        self._visualize_btree(dot, self.root)
        dot.render('btree.png', view=True)

    def _visualize_btree(self, dot, node):
        if node is not None:
            node_label = ', '.join(map(str, node.keys))
            dot.node(node_label, node_label, shape='box')
            if not node.leaf:
                for child in node.children:
                    child_label = ', '.join(map(str, child.keys))
                    dot.node(child_label, child_label, shape='box')
                    dot.edge(node_label, child_label)
                    self._visualize_btree(dot, child)

# Example usage
btree = BTree(2)
keys = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
for key in keys:
    btree.insert(key)
btree.visualize()


### Code Output:

![btree](https://maycooper.github.io/E-Commerce-PostgreSQL-Project/btree.png)


### Here's a simpler visualisation: 

The B-tree starts with a root node and may have multiple levels of nodes below it, forming a tree-like structure. The number of keys and children in each node is typically determined by a parameter called the "order" of the tree. Higher-order B-trees can store more keys and children in each node, which can lead to more efficient operations.

In [14]:
class BTreeNode:
    def __init__(self, keys=None, children=None):
        self.keys = keys or []
        self.children = children or []

    def __repr__(self):
        return f"BTreeNode({self.keys}, {self.children})"

def visualize_btree(node, level=0):
    """
    Visualizes a B-tree by printing the keys in each node and recursively
    visualizing its children. Adds indentation based on the level of the node.
    """
    if node is not None:
        print("  " * level + str(node.keys))
        for child in node.children:
            visualize_btree(child, level + 1)

# Create a sample B-tree
# B-tree with root node having keys [50, 80]
# First level children with varying number of keys
# Second level grandchildren with varying number of keys
btree = BTreeNode([50, 80], [
    BTreeNode([20, 30, 40], [
        BTreeNode([10], [
            BTreeNode([5], []),
            BTreeNode([15], [])
        ]),
        BTreeNode([25], []),
        BTreeNode([35], [
            BTreeNode([32], []),
            BTreeNode([38], [])
        ]),
        BTreeNode([45], [])
    ]),
    BTreeNode([60, 70], [
        BTreeNode([55], []),
        BTreeNode([65], []),
        BTreeNode([75], [])
    ]),
    BTreeNode([90, 100, 110], [
        BTreeNode([85], []),
        BTreeNode([95], []),
        BTreeNode([105], []),
        BTreeNode([115], [])
    ])
])

# Visualize the B-tree
visualize_btree(btree)


[50, 80]
  [20, 30, 40]
    [10]
      [5]
      [15]
    [25]
    [35]
      [32]
      [38]
    [45]
  [60, 70]
    [55]
    [65]
    [75]
  [90, 100, 110]
    [85]
    [95]
    [105]
    [115]


# Hash Indexing
### f(x) = y

Hash indexing uses a hash function to map data of arbitrary length to a fixed-size value, creating a unique "fingerprint" of the input data. Hash values are designed to be different for different inputs, and even slight changes in input produce different hash values. 

However, occasional collisions where different inputs produce the same hash value can occur, although they are rare. Hash indexes are used for equality comparisons and can be smaller and faster than B-tree indexes, making them advantageous for in-memory storage and fast data retrieval operations.

- Hash values are generally unique: Hash functions are designed to produce unique hash values for different inputs, minimizing the chances of collisions where two different inputs produce the same hash value. 
- Slight changes create a new hash: Hash functions are sensitive to changes in input data, even slight changes like adding or modifying a single character. 
- Size of hash depends on algo used: Some hash functions produce fixed-size hash values, while others may have variable-size hash values. 
- Ordering not preserved: Hash indexes are not suitable for range queries or other types of comparisons that require ordering of data.
- Hash indexes are used for equality comparisons: Hash indexes are typically used for performing equality comparisons, where the hash value of a search key is compared with the hash values of the data elements in the hash index to quickly locate a match. 
- Slight changes in input produce different hashes: This means that even a small change in the input data will likely result in a completely different hash value, providing a high level of data integrity and accuracy.

## Let's illustrate a hypothetical hash
### We'll illustrate a hash for the words "Hello, world! Carpe diem!"

In [19]:
import hashlib

def visualize_hash(data):
    # Create a hash object using hashlib
    hash_object = hashlib.md5(data.encode())
    # Get the hexadecimal representation of the hash
    hex_digest = hash_object.hexdigest()
    # Create a string to store the visual representation
    visual_repr = ""
    # Define a mapping of hexadecimal characters to ASCII art
    ascii_art_mapping = {
        '0': '   \n  |\n  |\n  |\n  |',
        '1': '   \n  |\n  |\n  |\n _|_',
        '2': ' _ \n  |\n _|\n| \n|_ ',
        '3': ' _ \n  |\n _|\n  |\n _|',
        '4': '   \n| |\n|_|\n  |\n  |',
        '5': ' _ \n|_ \n _|\n  |\n _|',
        '6': ' _ \n|_ \n|_|\n| |\n|_|',
        '7': ' _ \n  |\n  |\n  |\n  |',
        '8': ' _ \n|_|\n|_|\n|_|\n|_|',
        '9': ' _ \n|_|\n|_|\n  |\n _|',
        'a': '   \n _ \n|_|\n| |\n|_|',
        'b': '   \n|_ \n|_|\n| |\n|_|',
        'c': '   \n _ \n|  \n|  \n|_ ',
        'd': '   \n  |\n _| \n| |\n|_|',
        'e': '   \n _ \n|_ \n|_|\n|_ ',
        'f': '   \n _ \n|_ \n|_|\n|  ',
    }
    # Loop through each character in the hexadecimal representation
    for char in hex_digest:
        # Convert the character to lowercase
        char = char.lower()
        # If the character is a valid hexadecimal character, append its corresponding ASCII art to the visual representation
        if char in ascii_art_mapping:
            visual_repr += ascii_art_mapping[char] + " "
        else:
            visual_repr += "     "  # If the character is not valid, append a blank space
    # Return the visual representation as a string
    return visual_repr

# Example usage
data = "Hello, world! Carpe diem!"  # Input data to be hashed
visual_repr = visualize_hash(data)
print(visual_repr)


 _ 
  |
 _|
  |
 _|  _ 
  |
 _|
  |
 _|    
| |
|_|
  |
  |    
| |
|_|
  |
  |  _ 
  |
 _|
  |
 _|  _ 
|_ 
|_|
| |
|_|  _ 
|_|
|_|
|_|
|_|    
| |
|_|
  |
  |  _ 
  |
  |
  |
  |    
 _ 
|  
|  
|_     
  |
 _| 
| |
|_|  _ 
|_|
|_|
  |
 _|    
 _ 
|_ 
|_|
|    _ 
  |
 _|
| 
|_     
  |
 _| 
| |
|_|    
 _ 
|  
|  
|_     
 _ 
|  
|  
|_   _ 
|_ 
 _|
  |
 _|  _ 
|_ 
 _|
  |
 _|  _ 
  |
 _|
  |
 _|  _ 
  |
  |
  |
  |    
 _ 
|_|
| |
|_|  _ 
  |
 _|
| 
|_   _ 
  |
  |
  |
  |    
 _ 
|_|
| |
|_|    
| |
|_|
  |
  |  _ 
|_|
|_|
  |
 _|  _ 
|_|
|_|
|_|
|_|  _ 
  |
 _|
| 
|_   _ 
  |
  |
  |
  |    
  |
 _| 
| |
|_|    
 _ 
|  
|  
|_  


### Let's create a numeric represenation of the hash index for the words "Hello, world! Let's seize the day!" 

In [21]:
import hashlib

def visualize_hash(data):
    # Create a hash object using hashlib
    hash_object = hashlib.md5(data.encode())
    # Get the hexadecimal representation of the hash
    hex_digest = hash_object.hexdigest()
    # Create a list to store the visual representation
    visual_repr = []
    # Loop through each character in the hexadecimal representation
    for char in hex_digest:
        # Convert the character to its corresponding ASCII value
        ascii_value = ord(char)
        # Add the ASCII value to the list
        visual_repr.append(ascii_value)
    # Return the visual representation as a list of ASCII values
    return visual_repr

# Example usage
data = "Hello, world! Let's seize the day!"  # Input data to be hashed
visual_repr = visualize_hash(data)
print(visual_repr)


[102, 99, 101, 49, 53, 99, 52, 99, 99, 49, 54, 99, 98, 102, 99, 57, 102, 98, 102, 100, 99, 49, 51, 99, 100, 55, 98, 53, 57, 57, 49, 57]


## Using B-Tree for Indexing


In [ ]:
#SQL code for postgres pgadmin 
CREATE INDEX idx_lname_fname
ON customers USING btree
(last_name, first_name)

## Using Hash for Indexing

In [ ]:
#SQL code for postgres pgadmin 
CREATE INDEX idx_lname_fname
ON customers USING hash
(last_name, first_name)